<a href="https://colab.research.google.com/github/Kaiziferr/-Miner_Detector/blob/master/Sequential/03_workshop_parameter_optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
import pandas as pd
import numpy as np

In [4]:
def create_model():
  model = Sequential()
  model.add(Dense(12, input_dim = 8, activation='relu'))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  return model

In [5]:
url = 'https://raw.githubusercontent.com/Kaiziferr/Deep_Learning_Workshop/master/workshop_keras_sklearn_introduction/diabetes.csv'
data = pd.read_csv(url)
data_values = data.values

In [6]:
X = data_values[:,0:8]
y = data_values[:,8]

In [7]:
#wrapper
model = KerasClassifier(build_fn=create_model, epochs=150, batch_size = 10, verbose=0)

In [8]:
# StratifiedKFold conserva la distribución de clases del conjunto de datos en las divisiones
# Recuerda el concepto de startificado
kfold = StratifiedKFold(n_splits=10, shuffle=True)
result = cross_val_score(model, X, y, cv = kfold)

In [9]:
result

array([0.77922076, 0.64935064, 0.75324672, 0.8051948 , 0.6883117 ,
       0.70129871, 0.75324672, 0.77922076, 0.75      , 0.64473683])

In [10]:
result.mean()

0.730382764339447

# Optimización de hiperparametros

In [11]:
from sklearn.model_selection import GridSearchCV

In [13]:
def create_model_optimizer(optimizer = 'rmsprop', init = 'glorot_uniform'):
  model = Sequential()
  model.add(Dense(12, input_dim = 8, kernel_initializer=init, activation='relu'))
  model.add(Dense(8, kernel_initializer=init, activation='relu'))
  model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
  model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
  return model

In [16]:
model = KerasClassifier(build_fn=create_model_optimizer, verbose = 0)

In [26]:
optimizer = ['rmsprop', 'adam']
inits = ['glorot_uniform', 'normal', 'uniform']
epochs = [50,100,150]
batches = [5,10,20]
param_grid = dict(optimizer = optimizer, epochs = epochs, batch_size = batches, init = inits)
grid = GridSearchCV(estimator=model,param_grid=param_grid, cv = 3)

In [30]:
grid_result = grid.fit(X,y)

In [31]:
print("Mejor: %f usando %s" % (grid_result.best_score_, grid_result.best_params_)) 

Mejor: 0.748698 usando {'batch_size': 5, 'epochs': 150, 'init': 'uniform', 'optimizer': 'rmsprop'}


In [33]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
  print("%f (%f) con %r" % (mean, stdev, param) )

0.703125 (0.025516) con {'batch_size': 5, 'epochs': 50, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.686198 (0.008027) con {'batch_size': 5, 'epochs': 50, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.720052 (0.025780) con {'batch_size': 5, 'epochs': 50, 'init': 'normal', 'optimizer': 'rmsprop'}
0.707031 (0.008438) con {'batch_size': 5, 'epochs': 50, 'init': 'normal', 'optimizer': 'adam'}
0.705729 (0.024774) con {'batch_size': 5, 'epochs': 50, 'init': 'uniform', 'optimizer': 'rmsprop'}
0.694010 (0.015733) con {'batch_size': 5, 'epochs': 50, 'init': 'uniform', 'optimizer': 'adam'}
0.696615 (0.019225) con {'batch_size': 5, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.734375 (0.033603) con {'batch_size': 5, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.735677 (0.025976) con {'batch_size': 5, 'epochs': 100, 'init': 'normal', 'optimizer': 'rmsprop'}
0.743490 (0.018136) con {'batch_size': 5, 'epochs': 100, 'init': 'normal', 'optimizer': 'adam